# Deep Learning Playground

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [ ]:
%reload_ext autoreload
%autoreload 2

import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar

from src.acnets.deep import ACNetsDataModule, MultiHeadModel

In [20]:
datamodule = ACNetsDataModule(atlas='dosenbach2010', kind='partial correlation', batch_size=8)
datamodule.setup()

n_regions = datamodule.train[0][0].shape[1]
n_networks = datamodule.train[0][2].shape[1]

# DEBUG single epoch, one train/test split
model = MultiHeadModel(n_regions, n_networks, n_embeddings=32)
trainer = pl.Trainer(accelerator='auto',
                     max_epochs=10000,
                     accumulate_grad_batches=3,
                     gradient_clip_val=.5,
                     log_every_n_steps=3,
                     callbacks=[RichProgressBar()])
trainer.fit(model, datamodule)
trainer.test(model, datamodule=datamodule)

(160, 6)